In [1]:
import torch
import torch.nn as nn
import time

# --- 모델 정의 ---

# 1. 표준 GRU (수식 기반 직접 구현)
class StandardGRU(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim

        # 입력 x_t와 이전 은닉 상태 h_{t-1}를 합친 크기
        combined_dim = input_dim + hidden_dim

        # W_r, W_z, W_n 에 해당하는 Linear 레이어들
        self.linear_rz = nn.Linear(combined_dim, 2 * hidden_dim) # 리셋(r), 업데이트(z) 게이트 동시 계산
        self.linear_n = nn.Linear(combined_dim, hidden_dim)    # 후보(n) 상태 계산용

    def forward(self, x, h_prev=None):
        # x: (batch_size, seq_len, input_dim)
        batch_size, seq_len, _ = x.shape

        # 초기 은닉 상태 (없으면 0으로 초기화)
        if h_prev is None:
            h_prev = torch.zeros(batch_size, self.hidden_dim).to(x.device)

        outputs = [] # 각 타임스텝의 은닉 상태를 저장할 리스트

        # 시퀀스를 한 스텝씩 처리
        for t in range(seq_len):
            xt = x[:, t, :] # 현재 타임스텝 입력: (batch_size, input_dim)
            combined_input = torch.cat([h_prev, xt], dim=1) # (batch_size, hidden_dim + input_dim)

            # 리셋(r) 및 업데이트(z) 게이트 계산
            rz = self.linear_rz(combined_input)
            r_t, z_t = torch.chunk(rz, 2, dim=1) # 결과를 반으로 나눠 r, z 얻음
            r_t = torch.sigmoid(r_t) # 리셋 게이트: (batch_size, hidden_dim)
            z_t = torch.sigmoid(z_t) # 업데이트 게이트: (batch_size, hidden_dim)

            # 후보(n) 은닉 상태 계산
            # n_t = tanh(W_n * [r_t ⊙ h_{t-1}, x_t] + b_n)
            combined_n = torch.cat([r_t * h_prev, xt], dim=1) # 리셋 게이트 적용된 h와 x 결합
            n_t = torch.tanh(self.linear_n(combined_n)) # (batch_size, hidden_dim)

            # 최종 은닉 상태(h) 계산
            # h_t = (1 - z_t) ⊙ n_t + z_t ⊙ h_{t-1}
            h_t = (1 - z_t) * n_t + z_t * h_prev # (batch_size, hidden_dim)

            outputs.append(h_t.unsqueeze(1)) # (batch_size, 1, hidden_dim)
            h_prev = h_t # 다음 스텝을 위해 현재 은닉 상태 저장

        # 모든 타임스텝의 출력을 모음
        outputs = torch.cat(outputs, dim=1) # (batch_size, seq_len, hidden_dim)
        # 마지막 은닉 상태와 모든 시퀀스 출력을 반환
        return outputs, h_prev # h_prev는 마지막 타임스텝의 h_t와 동일



In [2]:
# 2. Minimal GRU (minGRU) (수식 기반 직접 구현)
class MinimalGRU(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim

        # 입력 x_t와 이전 은닉 상태 h_{t-1}를 합친 크기
        combined_dim = input_dim + hidden_dim

        # W_f, W_h 에 해당하는 Linear 레이어들
        self.linear_f = nn.Linear(combined_dim, hidden_dim) # 망각(f) 게이트 계산용
        self.linear_h = nn.Linear(combined_dim, hidden_dim) # 후보(h̃) 상태 계산용

    def forward(self, x, h_prev=None):
        # x: (batch_size, seq_len, input_dim)
        batch_size, seq_len, _ = x.shape

        # 초기 은닉 상태 (없으면 0으로 초기화)
        if h_prev is None:
            h_prev = torch.zeros(batch_size, self.hidden_dim).to(x.device)

        outputs = []

        for t in range(seq_len):
            xt = x[:, t, :]
            combined_input = torch.cat([h_prev, xt], dim=1)

            # 망각(f) 게이트 계산
            # f_t = σ(W_f * [h_{t-1}, x_t] + b_f)
            f_t = torch.sigmoid(self.linear_f(combined_input)) # (batch_size, hidden_dim)

            # 후보(h̃) 은닉 상태 계산
            # h̃_t = tanh(W_h * [f_t ⊙ h_{t-1}, x_t] + b_h)
            combined_h = torch.cat([f_t * h_prev, xt], dim=1) # 망각 게이트 적용된 h와 x 결합
            h_tilde = torch.tanh(self.linear_h(combined_h)) # (batch_size, hidden_dim)

            # 최종 은닉 상태(h) 계산 (구현 방식 2: h_t = f_t * h_{t-1} + (1 - f_t) * h̃_t)
            h_t = f_t * h_prev + (1 - f_t) * h_tilde # (batch_size, hidden_dim)

            outputs.append(h_t.unsqueeze(1))
            h_prev = h_t

        outputs = torch.cat(outputs, dim=1)
        return outputs, h_prev

In [ ]:

# --- 비교 실행 ---
# 파라미터 설정
input_dim = 50
hidden_dim = 100
seq_len = 30
batch_size = 4

# 모델 인스턴스화
std_gru_manual = StandardGRU(input_dim, hidden_dim)
min_gru = MinimalGRU(input_dim, hidden_dim)
# 참고: PyTorch 내장 GRU (비교용)
std_gru_builtin = nn.GRU(input_dim, hidden_dim, batch_first=True)

# 파라미터 수 계산 함수
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# 각 모델의 파라미터 수 출력
params_std_manual = count_parameters(std_gru_manual)
params_min = count_parameters(min_gru)
params_std_builtin = count_parameters(std_gru_builtin)

print(f"--- 모델 파라미터 수 비교 ---")
print(f"Standard GRU (Manual): {params_std_manual:,}")
print(f"Minimal GRU (minGRU): {params_min:,}")
print(f"Standard GRU (Built-in): {params_std_builtin:,}")
print("-" * 20)
# 참고: 직접 구현한 Standard GRU와 내장 GRU의 파라미터 수가 정확히 일치하는 것을 볼 수 있습니다.
# minGRU는 게이트가 줄어 파라미터 수가 더 적습니다. (표준 GRU의 약 2/3)

# 더미 입력 데이터 생성
dummy_input = torch.randn(batch_size, seq_len, input_dim)

# 모델 실행 및 출력 확인 (시간 측정 포함)
print(f"--- 모델 실행 시간 및 출력 형태 비교 ---")
start_time = time.time()
outputs_std_manual, hn_std_manual = std_gru_manual(dummy_input)
print(f"Standard GRU (Manual) - 실행 시간: {time.time() - start_time:.4f}s")
print(f"  Output shape: {outputs_std_manual.shape}") # (batch_size, seq_len, hidden_dim)
print(f"  Hidden shape: {hn_std_manual.shape}")     # (batch_size, hidden_dim)

start_time = time.time()
outputs_min, hn_min = min_gru(dummy_input)
print(f"Minimal GRU (minGRU) - 실행 시간: {time.time() - start_time:.4f}s")
print(f"  Output shape: {outputs_min.shape}")
print(f"  Hidden shape: {hn_min.shape}")

start_time = time.time()
outputs_std_builtin, hn_std_builtin = std_gru_builtin(dummy_input)
print(f"Standard GRU (Built-in) - 실행 시간: {time.time() - start_time:.4f}s")
print(f"  Output shape: {outputs_std_builtin.shape}")
print(f"  Hidden shape: {hn_std_builtin.shape}") # Built-in은 (num_layers, batch_size, hidden_dim) 형태
print("-" * 20)

--- 모델 파라미터 수 비교 ---
Standard GRU (Manual): 45,300
Minimal GRU (minGRU): 30,200
Standard GRU (Built-in): 45,600
--------------------
--- 모델 실행 시간 및 출력 형태 비교 ---
Standard GRU (Manual) - 실행 시간: 0.0113s
  Output shape: torch.Size([4, 30, 100])
  Hidden shape: torch.Size([4, 100])
Minimal GRU (minGRU) - 실행 시간: 0.0061s
  Output shape: torch.Size([4, 30, 100])
  Hidden shape: torch.Size([4, 100])
Standard GRU (Built-in) - 실행 시간: 0.0052s
  Output shape: torch.Size([4, 30, 100])
  Hidden shape: torch.Size([1, 4, 100])
--------------------
